# Analýza souboru GTF lidského genomu

## Filip Sedlák

## Analýza

Pomocí příkazu `grep` jsem rozdělil GTF soubor na geny a transkripty.

*Pozn.* Použitím `!` v IPython Notebooku lze spouštět bashové příkazy.

In [1]:
!cat Homo_sapiens.GRCh38.86.gtf | grep -E "\s+gene\s+" > Homo_sapiens.GRCh38.86_gene.gtf

In [2]:
!cat Homo_sapiens.GRCh38.86.gtf | grep -E "\s+transcript\s+" > Homo_sapiens.GRCh38.86_transcript.gtf

Naimportoval jsem potrebne moduly.

In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt
import pylab as pl
import re

Načetl jsem soubory s geny a transkripty, vytvořené z GTF souboru.

In [4]:
genes=pd.read_table('/home/nasta/Documents/python_bio/project/Homo_sapiens.GRCh38.86_gene.gtf', header=None, dtype={0:np.object})   
transcripts=pd.read_table('/home/nasta/Documents/python_bio/project/Homo_sapiens.GRCh38.86_transcript.gtf', header=None, dtype={0:np.object})        

In [5]:
genes.columns = ["seqname", "source", "feature", "start", "end", "score", "strand", "frame", "attribute"]
transcripts.columns = ["seqname", "source", "feature", "start", "end", "score", "strand", "frame", "attribute"]

In [6]:
genes.head()

,seqname,source,feature,start,end,score,strand,frame,attribute
0,1,havana,gene,11869,14409,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""5""; g..."
1,1,havana,gene,14404,29570,.,-,.,"gene_id ""ENSG00000227232""; gene_version ""5""; g..."
2,1,mirbase,gene,17369,17436,.,-,.,"gene_id ""ENSG00000278267""; gene_version ""1""; g..."
3,1,ensembl_havana,gene,29554,31109,.,+,.,"gene_id ""ENSG00000243485""; gene_version ""4""; g..."
4,1,havana,gene,34554,36081,.,-,.,"gene_id ""ENSG00000237613""; gene_version ""2""; g..."


Vytvořil jsem funkci pro rzdělení devátého sloupce - "attribute".

In [7]:
def parse_attributes(attributes_str):
    """Načte obsah sloupce attribute jako `dict`.
    
    - Pozor, neporadí si se středníky v hodnotách a očekává uvozovky
      kolem každé hodnoty.
      
      
    Parametry:
    
    `attributes_str` - hodnota GTF sloupce attribute jako string
    """
    ONLY_ATTRIBUTES = set(["gene_id", 
                          "transcript_id",
                          "gene_name",
                          "gene_biotype",
                          "transcript_name",
                          "transcript_biotype"])
    
    out = {}
    
    for pair in attributes_str.split(";"):
        if pair.strip() == "":
            continue
        m = re.match(r"^\s*(.+) \"(.+)\"$", pair)
        
        if m.group(1) in ONLY_ATTRIBUTES:
            out[m.group(1)] = m.group(2)

    return out


s = """gene_id "ENSG00000241860"; gene_version "6"; transcript_id "ENST00000484859"; transcript_version "1"; gene_name "RP11-34P13.13"; gene_source "havana"; gene_biotype "processed_transcript"; havana_gene "OTTHUMG00000002480"; havana_gene_version "3"; transcript_name "RP11-34P13.13-004"; transcript_source "havana"; transcript_biotype "antisense"; havana_transcript "OTTHUMT00000007035"; havana_transcript_version "1"; tag "basic"; transcript_support_level "5";"""
parse_attributes(s)

{'gene_biotype': 'processed_transcript',
 'gene_id': 'ENSG00000241860',
 'gene_name': 'RP11-34P13.13',
 'transcript_biotype': 'antisense',
 'transcript_id': 'ENST00000484859',
 'transcript_name': 'RP11-34P13.13-004'}

Takto jsem rozdělil sloupec v obou tabulkách.

In [8]:
def split_attribute_column(df):
    """Rozdělí sloupec `attribute` data frame GTF souboru do vlastních
    sloupců.
    """
    attributes_columns = df.attribute.apply(parse_attributes).apply(pd.Series)
    return pd.concat([df.drop(["attribute"], axis=1),
                      attributes_columns],
                     axis=1)

genes = split_attribute_column(genes)
transcripts = split_attribute_column(transcripts)

In [9]:
genes.head()

,seqname,source,feature,start,end,score,strand,frame,gene_biotype,gene_id,gene_name
0,1,havana,gene,11869,14409,.,+,.,transcribed_unprocessed_pseudogene,ENSG00000223972,DDX11L1
1,1,havana,gene,14404,29570,.,-,.,unprocessed_pseudogene,ENSG00000227232,WASH7P
2,1,mirbase,gene,17369,17436,.,-,.,miRNA,ENSG00000278267,MIR6859-1
3,1,ensembl_havana,gene,29554,31109,.,+,.,lincRNA,ENSG00000243485,MIR1302-2
4,1,havana,gene,34554,36081,.,-,.,lincRNA,ENSG00000237613,FAM138A


In [10]:
transcripts.head()

,seqname,source,feature,start,end,score,strand,frame,gene_biotype,gene_id,gene_name,transcript_biotype,transcript_id,transcript_name
0,1,havana,transcript,11869,14409,.,+,.,transcribed_unprocessed_pseudogene,ENSG00000223972,DDX11L1,processed_transcript,ENST00000456328,DDX11L1-002
1,1,havana,transcript,12010,13670,.,+,.,transcribed_unprocessed_pseudogene,ENSG00000223972,DDX11L1,transcribed_unprocessed_pseudogene,ENST00000450305,DDX11L1-001
2,1,havana,transcript,14404,29570,.,-,.,unprocessed_pseudogene,ENSG00000227232,WASH7P,unprocessed_pseudogene,ENST00000488147,WASH7P-001
3,1,mirbase,transcript,17369,17436,.,-,.,miRNA,ENSG00000278267,MIR6859-1,miRNA,ENST00000619216,MIR6859-1-201
4,1,havana,transcript,29554,31097,.,+,.,lincRNA,ENSG00000243485,MIR1302-2,lincRNA,ENST00000473358,MIR1302-2-001


Vybral jsem geny, které se nacházejí na standardních chromozomech.

In [11]:
genes["seqname"].unique()

array(['1', '2', '3', '4', '5', '6', '7', 'X', '8', '9', '11', '10', '12',
       '13', '14', '15', '16', '17', '18', '20', '19', 'Y', '22', '21',
       'MT', 'KI270728.1', 'KI270727.1', 'KI270442.1', 'GL000225.1',
       'GL000009.2', 'GL000194.1', 'GL000205.2', 'GL000195.1',
       'KI270733.1', 'GL000219.1', 'GL000216.2', 'KI270744.1',
       'KI270734.1', 'GL000213.1', 'GL000220.1', 'GL000218.1',
       'KI270731.1', 'KI270750.1', 'KI270721.1', 'KI270726.1',
       'KI270711.1', 'KI270713.1'], dtype=object)

In [12]:
CHROMOSOMES = ['1', '2', '3', '4', '5', '6', '7', '8', '9', 
               '11', '10', '12', '13', '14', '15', '16', 
               '17', '18', '20', '19', '22', '21',
               'X', 'Y', 
               'MT']

genes = genes[genes["seqname"].isin(CHROMOSOMES)]
transcripts = transcripts[transcripts["seqname"].isin(CHROMOSOMES)]

In [13]:
transcripts["seqname"].unique()

array(['1', '2', '3', '4', '5', '6', '7', 'X', '8', '9', '11', '10', '12',
       '13', '14', '15', '16', '17', '18', '20', '19', 'Y', '22', '21',
       'MT'], dtype=object)